## Creazione di un'istanza Flask

Innanzitutto importiamo la classe `Flask`.
```python
from flask import Flask
```

E poi usiamo questa classe per creare un oggetto `app` che rappresenta la nostra applicazione.
```python
app = Flask(__name__)
```

## Routing / Controllers

Poi possiamo iniziare a definire le "route", ovvero i path (percorsi) a cui vogliamo che la nostra applicazione risponda.

```python
@app.route("/")
def hello_world():
    return  'Hello, World!"'
```

In questo caso, `"/"` indica la *root* del sito.
Quindi se il nostro sito attualmente risponde all'URL `http://127.0.0.1:5000`, il singolo slash `/` indica la "pagina principale" o la root del sito: `http://127.0.0.1:5000/`.

Possiamo naturalmente definire un qualunque altro percorso:

```python
@app.route('/hello')
def hello():
    return 'Ciao, utente!'
```
Questa route risponderà all'URL `http://127.0.0.1:5000/hello`

## Route con variabili

Puoi aggiungere parti variabili a un URL contrassegnando le sezioni con `<nome_variabile>`. La tua funzione riceve quindi `variable_name` come argomento di una parola chiave.

```python
@app.route("/saluta/<name>")
def hello_name(name):
    return f"Hello, {escape(name)}!"
```

Facoltativamente, puoi utilizzare un convertitore per specificare il tipo di argomento come `<converter:variable_name>`.

```python
@app.route('/post/<int:post_id>')
def show_post(post_id):
    return f'Il post richiesto è {post_id} che è di tipo {escape(type(post_id))}'
```

### Converter disponibili

| Convertitore | Descrizione                                       |
| ------------ | ------------------------------------------------- |
| `string`     | (default) accetta qualsiasi testo senza slash `/` |
| `int`        | accetta solo numeri interi positivi               |
| `float`      | accetta solo valori in virgola mobile positivi    |
| `path`       | come stringa ma accetta anche gli slash `/`       |
| `uuid`       | accetta stringhe in formato UUID                  |

## Escaping

Bisogna sempre prestare attenzione ai dati che arrivano dal mondo esterno, come quelli inseriti dagli utenti. Se non opportunamente trattati, questi dati potrebbero contenere codice HTML e/o Javascript. Dobbiamo dunque evitare che tali codici vengano "iniettati" nelle nostre pagine, perché potrebbero essere veicolo di codici malevoli.

Innanzitutto dobbiamo importare la funzione che ci consente di fare l'escape:
```python
from markupsafe import escape
```

Il seguente codice gestisce in modo corretto l'input proveniente dall'URL:
```python
@app.route('/path/<path:subpath>')
def show_subpath(subpath):
    # Ora abbiamo ciò che l'utente ha scritto dopo /path/
    return f'Il subpath è: {escape(subpath)}' 
```

Invece quest'altro non fa l'escape della stringa provenienete dall'URL, esponendo il sito a possibili attacchi:

```python
@app.route('/path_no_escape/<path:subpath>')
def show_subpath_not_escaped(subpath):
    return f'Il subpath è: {subpath}'
```

In questo caso, nel quale non viene utilizzato l'escape, se provassimo ad aprire i seguenti URL, otterremmo una iniezione di codice Javascript:

[`http://127.0.0.1:5000/path_no_escape/<script>alert("Hacked!")</script>`](http://127.0.0.1:5000/path_no_escape/<script>alert("Hacked!")</script>)

oppure

[`http://127.0.0.1:5000/path_no_escape/%3Cscript%3Ealert(%22Hacked!%22)%3C/script%3E`](http://127.0.0.1:5000/path_no_escape/%3Cscript%3Ealert(%22Hacked!%22)%3C/script%3E)

### Come fa l'escape a proteggerci?

In pratica converte i caratteri considerati "pericolosi" nei corrispettivi caratteri scritti usando la codifica HTML.

Per esempio, facendo l'escape, il codice HTML e Javascript presente nell'URL precedente:

```html
<script>alert("Hacked!")</script>
```

verrebbe convertito in:

```html
&lt;script&gt;alert(&#34;Hacked!&#34;)&lt;/script&gt;
```

Rendendolo così inoffensivo.


## Scrivere caratteri speciali usando le codifiche web standard

Il sito [amp-what.com](https://www.amp-what.com/) è molto utile per ottenere la codifica di tutti i caratteri di cui potremmo avere necessità.

Il carattere `<` può essere codificato in vari modi:

- `%3C` usando la codifica per gli URL (%N)

- `&#60;` usando la codifica delle entità HTML in decimale (base 10)

- `&#x3c;` usando la codifica delle entità HTML in esadecimale (base 16)

- `&lt;` usando la codifica delle entità HTML "nominate" (*named*)

La codifica che usa i nomi anziché i codici numerici è più facile da ricordare.

Ecco alcuni dei più comuni simboli che possiamo incontrare e di cui possiamo avere bisogno:

- `&lt;` rappresenta il simbolo `<` (lower-than)

- `&gt;` rappresenta il simbolo `>` (greater-than)

- `&amp;` rappresenta il simbolo `&` (ampersand)

- `&quot;` rappresenta il simbolo di virgolette `"` (quote)

- `&apos;` rappresenta il simbolo di apice singolo `'` (apostrophe)

- `&nbsp;` rappresenta uno spazio speciale "indivisibile" (non-breaking space)

## Utilizzo degli strumenti per sviluppatori del browser

 Gli strumenti per sviluppatori che ci serviranno maggiormente sono (con riferimento al browser Google Chrome):

 - **Elements** (analizziamo la struttura della pagina)

 - **Console** (controlliamo se ci sono errori Javascritp)

 - **Sources** (vediamo tutte le risorse caricate dalla pagina)

 - **Network** (vediamo tutte le richieste avvenute per scaricare le risorse richieste dalla pagina). In particolare questa sezione è molto utile per leggere gli *header* della richiesta e della risposta e inoltre il contenuto effettivo della risposta.